Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [44]:
batch_size = 128
beta_L2 = 5e-4

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
#   tf_train_dataset = tf.placeholder(tf.float32,
#                                     shape=(batch_size, image_size * image_size))
#   tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_dataset = tf.placeholder(tf.float32) #should have size (batch_size, image_size * image_size)
  tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu1 = tf.nn.relu(logits1)
  L2_reg1 = tf.nn.l2_loss(weights1)
  logits2 = tf.matmul(relu1, weights2) + biases2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  L2_reg2 = tf.nn.l2_loss(weights2)
  loss_reg = loss + beta_L2 * (L2_reg1+L2_reg2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_reg)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [40]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss_reg, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 507.232910
Minibatch accuracy: 12.0%
Validation accuracy: 34.9%
Minibatch loss at step 500: 129.617859
Minibatch accuracy: 86.0%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 103.758484
Minibatch accuracy: 81.0%
Validation accuracy: 79.5%
Minibatch loss at step 1500: 75.743393
Minibatch accuracy: 86.0%
Validation accuracy: 78.7%
Minibatch loss at step 2000: 56.291687
Minibatch accuracy: 83.0%
Validation accuracy: 81.5%
Minibatch loss at step 2500: 44.447380
Minibatch accuracy: 82.0%
Validation accuracy: 82.3%
Minibatch loss at step 3000: 33.908253
Minibatch accuracy: 83.0%
Validation accuracy: 83.2%
Test accuracy: 90.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

In [46]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized, using batch size: %s" % batch_size)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    idx = np.random.randint(train_dataset.shape[0]/100, size=batch_size)
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    batch_data = train_dataset[idx]
    batch_labels = train_labels[idx]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss_reg, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Updated batch size: %s" % batch_size)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized, using batch size: 128
Updated batch size: 128
Minibatch loss at step 0: 417.093292
Minibatch accuracy: 14.8%
Validation accuracy: 27.8%
Updated batch size: 128
Minibatch loss at step 500: 122.226471
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Updated batch size: 128
Minibatch loss at step 1000: 95.147011
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Updated batch size: 128
Minibatch loss at step 1500: 74.079308
Minibatch accuracy: 100.0%
Validation accuracy: 79.0%
Updated batch size: 128
Minibatch loss at step 2000: 57.685619
Minibatch accuracy: 100.0%
Validation accuracy: 78.8%
Updated batch size: 128
Minibatch loss at step 2500: 44.922989
Minibatch accuracy: 100.0%
Validation accuracy: 78.6%
Updated batch size: 128
Minibatch loss at step 3000: 34.982975
Minibatch accuracy: 100.0%
Validation accuracy: 78.5%
Test accuracy: 86.3%


---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [70]:
batch_size = 128
beta_L2 = 5e-4
dropout_p = 1

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
#   tf_train_dataset = tf.placeholder(tf.float32,
#                                     shape=(batch_size, image_size * image_size))
#   tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_train_dataset = tf.placeholder(tf.float32) #should have size (batch_size, image_size * image_size)
  tf_train_labels = tf.placeholder(tf.float32,shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, 1024]))
  biases1 = tf.Variable(tf.zeros([1024]))
  weights2 = tf.Variable(tf.truncated_normal([1024, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu1 = tf.nn.dropout(tf.nn.relu(logits1), dropout_p) # with dropout
  L2_reg1 = tf.nn.l2_loss(weights1)
  logits2 = tf.matmul(relu1, weights2) + biases2
  L2_reg2 = tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  loss_reg = loss + beta_L2 * (L2_reg1+L2_reg2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_reg)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [71]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss_reg, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 420.956787
Minibatch accuracy: 14.8%
Validation accuracy: 28.9%
Minibatch loss at step 500: 129.525894
Minibatch accuracy: 78.9%
Validation accuracy: 80.0%
Minibatch loss at step 1000: 97.305847
Minibatch accuracy: 84.4%
Validation accuracy: 80.2%
Minibatch loss at step 1500: 75.290184
Minibatch accuracy: 80.5%
Validation accuracy: 81.8%
Minibatch loss at step 2000: 56.795609
Minibatch accuracy: 77.3%
Validation accuracy: 80.3%
Minibatch loss at step 2500: 44.682629
Minibatch accuracy: 84.4%
Validation accuracy: 83.1%
Minibatch loss at step 3000: 33.829151
Minibatch accuracy: 89.8%
Validation accuracy: 84.5%
Test accuracy: 90.9%


In [53]:
#train with small samples resulting in overfitting, even using dropout
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized, using batch size: %s" % batch_size)
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    #offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    idx = np.random.randint(train_dataset.shape[0]/100, size=batch_size)
    # Generate a minibatch.
    #batch_data = train_dataset[offset:(offset + batch_size), :]
    #batch_labels = train_labels[offset:(offset + batch_size), :]
    batch_data = train_dataset[idx]
    batch_labels = train_labels[idx]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss_reg, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Updated batch size: %s" % batch_size)
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized, using batch size: 128
Updated batch size: 128
Minibatch loss at step 0: 423.330566
Minibatch accuracy: 12.5%
Validation accuracy: 36.5%
Updated batch size: 128
Minibatch loss at step 500: 122.049370
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Updated batch size: 128
Minibatch loss at step 1000: 95.012154
Minibatch accuracy: 100.0%
Validation accuracy: 78.9%
Updated batch size: 128
Minibatch loss at step 1500: 73.981323
Minibatch accuracy: 100.0%
Validation accuracy: 79.3%
Updated batch size: 128
Minibatch loss at step 2000: 57.607471
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Updated batch size: 128
Minibatch loss at step 2500: 44.862026
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Updated batch size: 128
Minibatch loss at step 3000: 34.935268
Minibatch accuracy: 100.0%
Validation accuracy: 78.7%
Test accuracy: 86.3%


---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---



In [77]:
# SGD for speed,
# Regulization&Dropout against overfitting, 
# Multi-Layer

batch_size = 128
beta_L2 = 5e-4
dropout_p = 1
n_hidden_relu1 = 1024
n_hidden_relu2 = 1024
#n_hidden_relu3 = 1024

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(tf.truncated_normal([image_size * image_size, n_hidden_relu1]))
  biases1 = tf.Variable(tf.zeros([n_hidden_relu1]))
  weights2 = tf.Variable(tf.truncated_normal([n_hidden_relu1, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
#   weights3 = tf.Variable(tf.truncated_normal([n_hidden_relu2, num_labels]))
#   biases3 = tf.Variable(tf.zeros([num_labels]))

  # Training computation.
  # Layer 1  
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu1 = tf.nn.dropout(tf.nn.relu(logits1), dropout_p) # with dropout
  L2_reg1 = tf.nn.l2_loss(weights1)
    
  # Layer 2
  logits2 = tf.matmul(relu1, weights2) + biases2
#   relu2 = tf.nn.dropout(tf.nn.relu(logits2), dropout_p)
  L2_reg2 = tf.nn.l2_loss(weights2)
    
  # Layer 3
#   logits3 = tf.matmul(relu2, weights3) + biases3
#   L2_reg3 = tf.nn.l2_loss(weights3)

  
  # calculate the total loss
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits2, tf_train_labels))
  loss_reg = loss + beta_L2 * (L2_reg1+L2_reg2)
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss_reg)
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.5, global_step, 10000, 0.96)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss_reg, global_step=global_step)
    

  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)
#   valid_prediction = tf.nn.softmax(
#     tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3)
#   test_prediction = tf.nn.softmax(
#     tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2), weights3) + biases3)
  valid_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1), weights2) + biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1), weights2) + biases2)

In [80]:
num_steps = 20001

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss_reg, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 451.349915
Minibatch accuracy: 8.6%
Validation accuracy: 27.6%
Minibatch loss at step 500: 129.386414
Minibatch accuracy: 74.2%
Validation accuracy: 81.4%
Minibatch loss at step 1000: 98.160515
Minibatch accuracy: 79.7%
Validation accuracy: 81.2%
Minibatch loss at step 1500: 74.769516
Minibatch accuracy: 79.7%
Validation accuracy: 81.5%
Minibatch loss at step 2000: 57.763535
Minibatch accuracy: 79.7%
Validation accuracy: 80.0%
Minibatch loss at step 2500: 44.406734
Minibatch accuracy: 84.4%
Validation accuracy: 83.0%
Minibatch loss at step 3000: 34.076214
Minibatch accuracy: 90.6%
Validation accuracy: 83.5%
Minibatch loss at step 3500: 26.622906
Minibatch accuracy: 91.4%
Validation accuracy: 85.4%
Minibatch loss at step 4000: 21.163061
Minibatch accuracy: 82.0%
Validation accuracy: 85.9%
Minibatch loss at step 4500: 16.382696
Minibatch accuracy: 93.8%
Validation accuracy: 86.0%
Minibatch loss at step 5000: 13.017724
Minibatch accuracy: 89.8%
Valida

In [ ]:
# Best model record is 94.8%